In [2]:
#%gui tk
#from netgen import gui
from netgen.geom2d import unit_square
from netgen import geom2d
from netgen.meshing import *

In [3]:
# global mesh
msh = unit_square.GenerateMesh(maxh=.2)

In [4]:
# map each vertex to a list of its adjacent elements
def vmap(msh):
    vs = {}
    for e in msh.Elements2D():
        for v in e.vertices:
            es = vs.get(v.nr)
            if es:
                es.append(e)
            else:
                vs[v.nr] = [e]
    return vs    

In [5]:
# return a set of the unique vertices for each vertex patch
def vsets(vmap):
    vsets_dict = {}
    for v in vmap.keys():
        vset = set([v1 for e in vmap[v] for v1 in e.vertices])
        vsets_dict[v] = vset
    return vsets_dict


In [6]:
# takes a list of points representing the boundary of the vertex patch
# returns a geometry
def make_geom(bnd):
    geo = geom2d.SplineGeometry()
    ps = [geo.AppendPoint(*v) for v in bnd]
    for i in range(len(ps)-1):
        geo.Append (["line", ps[i],ps[i+1]])
    geo.Append(["line", ps[-1],ps[0]])
    return geo


In [7]:
# takes a list of 'elements' where each 'element' is an ordered list of vertex nrs
# the first n-1 elements are triangles, the last is a polygon
# returns an ordered list of boundary vertices (polygon)
def make_boundary(els):
    if len(els) == 0: raise ArgumentError("make_boundary")
    if len(els) == 1: return els[0]
    poly = els.pop()
    e, v1, v2, rest = find_common(poly, els)
    merged = merge_two(poly,e,v1,v2)
    rest.append(merged)
    return make_boundary(rest)


In [8]:
# takes a polygon and a list of at least two triangles 
# and extracts the first triangle found which shares an edge with the polygon
def find_common(poly, els):
    e, v1, v2 = find_match(poly, els)
    els.remove(e)
    return e, v1, v2, els


In [9]:
# takes a polygon and a list of triangles, at least one of which has an edge in common with the given polygon
# returns a matching triangle along with the two common vertices
def find_match(poly, els):
    n = len(poly)
    for e in els:
        for i in range(n-1):
            if poly[i] in e and poly[i+1] in e:
                return e, poly[i], poly[i+1]
        if poly[0] in e and poly[-1] in e:
            return e, poly[0], poly[-1]
    raise ArgumentError("find_match")
  

In [10]:
# given two elements and their common vertices, 
# return a merged element representing the boundary of the pair
def merge_two(poly, e, v1, v2):
    i1 = poly.index(v1)
    i2 = poly.index(v2)
    e.remove(v1)
    e.remove(v2)
    if i2-i1 == 1: return poly[:i1+1]+e+poly[i2:]
    elif i1-i2 == 1: return poly[:i2+1]+e+poly[i1:]
    else: return poly + e


In [11]:
# given a list of elements in a vertex patch, 
# a set of the indices of all vertices in the patch 
# and the sorted list of points for the mesh 
# construct a new geometry and mesh for the vertex patch
def vpatch_mesh(els, vset, pts):
    # first make a list of sublists, each of which contains the vertex indices of one element
    vs = [[v.nr for v in el.vertices] for el in els]
    # construct a boundary polygon from this list
    bndnums = make_boundary(vs)
    # map the vertex numbers of the polygon to 3D points
    bndpts = [pts[v - 1] for v in bndnums]
    # make the local geometry
    geo = make_geom(bndpts)
    # create the local mesh
    msh2 = Mesh(dim=2)
    msh2.SetGeometry(geo)
    msh2.SetMaterial(1,"mat")
    ptmap = {} 
    # map global vertex numbers to local ones, while adding points to the mesh
    for v in vset:
        x,y,z = pts[v.nr - 1].p
        ptmap[v.nr] = msh2.Add(MeshPoint(Pnt(x,y,z)))
    
    for el in els:
        # map the old vertices to the new ones
        vs = [ptmap[v.nr] for v in el.vertices]
        # add each element to the mesh
        msh2.Add(Element2D(1, vs)) # crashes here
    return msh2
        

In [11]:
vm = vmap(msh)
vs = vsets(vm)
v = 13

els = vm[v]
vset = vs[v]
pts = [p for p in msh.Points()]


In [12]:
vs = [[v.nr for v in el.vertices] for el in els]
bndnums = make_boundary(vs)
bndpts = [pts[v - 1] for v in bndnums]
bndpts2d = [(x,y) for (x,y,_) in bndpts]
geo = make_geom(bndpts)
msh2 = Mesh(dim=2)
msh2.SetGeometry(geo)
ptmap = {} 
# map global vertex numbers to local ones, while adding points to the mesh
for v in vset:
    x,y,z = pts[v.nr - 1].p
    ptmap[v.nr] = msh2.Add(MeshPoint(Pnt(x,y,z)))


In [ ]:
for el in els:
    vs = [ptmap[v.nr] for v in el.vertices]
    print(vs)
for p in msh2.Points():
     x,y,z = p.p
     print ("x = ", x, "y = ", y)
msh2.SetMaterial(1,"mat")
msh2.Add(Element2D(1, [2,4,3]))


In [ ]:
for el in els:
    # map the old vertices to the new ones
    vs = [ptmap[v.nr] for v in el.vertices]
    # add each element to the mesh
    msh2.Add(Element2D(1, vs))
